<center><h2>Importing the Libraries</h2></center>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

<center><h2>Reading the Dataset</h2></center>

In [ ]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='NAASk6pRpwyrEjOrYAyvhS57VDikhORRt2nLpnhzqHZN',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'universityadmiteligibilitypredict-donotdelete-pr-zz591uvq0llvol'
object_key = 'Admission_Predict.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)
data.head()


<center><h2>Analyse the Data</h2><center>

In [ ]:
data.head()

In [ ]:
data.drop(["Serial No."],axis=1,inplace=True)
data.head()

<center><h2>Handling Missing Values</h2></center>

In [ ]:
data.isnull().sum()

<center><h2>Data Description</h2></center>

In [ ]:
data.describe()

In [ ]:
data.info()

<center><h2>Data Visualisation</h2></center>



In [ ]:
from scipy import stats

In [ ]:
corr_matrix = data.corr()
corr_matrix

In [ ]:
c=np.random.normal(loc=5,size=100,scale=2)

<small> HISTOGRAM </small>

In [ ]:
sns.histplot(data["GRE Score"],color='orange')


In [ ]:
sns.histplot(data["TOEFL Score"],color='violet')

In [ ]:
sns.displot(data["University Rating"],color='indigo')

In [ ]:
sns.boxplot(data["TOEFL Score"],color='brown')

<small> BOX PLOT </small>

In [ ]:
sns.boxplot(x=data['University Rating'],color='pink') 

In [ ]:
sns.histplot(data.CGPA,color="blue")

<small> COUNT PLOT </small>

In [ ]:
sns.countplot(x=data['TOEFL Score'])

###### BIVARIATE ANALYSIS
<hr style="height:1px;">

<small> BAR PLOT </small>

In [ ]:
sns.barplot(x=data.SOP,y=data.CGPA)

<small>LINEAR PLOT</small>

In [ ]:
sns.lineplot(data["GRE Score"],data["SOP"], color='aqua')

In [ ]:
sns.lineplot(x=data.SOP,y=data.CGPA, color='purple')

<small>SCATTER PLOT</small>

In [ ]:
sns.scatterplot(x=data['TOEFL Score'],y=data['University Rating'],color='green')

<small>POINT PLOT</small>

In [ ]:
sns.pointplot(x=data['SOP'],y=data['CGPA'],color='yellow')

<small>REG PLOT</small>

In [ ]:
sns.regplot(x=data.CGPA,y=data['GRE Score'],color='Brown') 

<small>RELPLOT</small>

In [ ]:
sns.relplot(x="GRE Score",y="CGPA",data=data)

<center><h2>SPLITTING DEPENDENT AND INDEPENDENT COLUMNS</h2></center>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [ ]:
x = data[["GRE Score","TOEFL Score","University Rating","SOP","LOR ","CGPA", "Research"]]
y = data["Chance of Admit "].values.reshape(-1,1)

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
multiple_lin_reg = LinearRegression()
multiple_lin_reg.fit(x_train,y_train)

y_pred_mlr = multiple_lin_reg.predict(x_test)

r2_score_mlr = r2_score(y_test,y_pred_mlr)
print("Mutiple Linear Regression's Score = {:.3f}".format(r2_score_mlr))

In [ ]:
tree_reg = DecisionTreeRegressor()
tree_reg.fit(x_train,y_train)

y_pred_tree = tree_reg.predict(x_test)

r2_score_tree = r2_score(y_test,y_pred_tree)
print("Decision Tree Regression's Score = {:.3f}".format(r2_score_tree))

In [ ]:
ran_for_reg = RandomForestRegressor(n_estimators=100,random_state=42)
ran_for_reg.fit(x_train,y_train)

y_pred_rfr = ran_for_reg.predict(x_test)

r2_score_rfr = r2_score(y_test,y_pred_rfr)
print("Random Forest Regression's Score = {:.3f}".format(r2_score_rfr))

<center><h2>SAVING THE MODELS</h2></center>

In [ ]:
import pickle
pickle.dump(multiple_lin_reg,open('university.pkl','wb'))
pickle.dump(multiple_lin_reg, open("university_percent.pkl", "wb"))
lr = pickle.load(open("university.pkl", "rb")) #logistic regression model
gd = pickle.load(open("university_percent.pkl", "rb")) #regression model

In [ ]:
!pip install ibm_watson_machine_learning

In [ ]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                    "url":"https://us-south.ml.cloud.ibm.com",
                    "apikey":"0zRs_HVV9HNSSzHy-sO_y_XWgsYQFlchHxmeakje1g0H"
                  }
client = APIClient(wml_credentials)

In [ ]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name']== space_name)['metadata']['id'])

In [ ]:
space_uid = guid_from_space_name(client,'models')
print("Space UID = " + space_uid)

In [ ]:
client.set.default_space(space_uid)

In [ ]:
client.software_specifications.list(100)

In [ ]:
software_spec_uid =client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

In [ ]:
model_details = client.repository.store_model(model=lr, meta_props={
     client.repository.ModelMetaNames.NAME: "University_admit",
     client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
     client.repository.ModelMetaNames.TYPE: "scikit-learn_1.0"}
                                             )


In [ ]:
model_id = client.repository.get_model_id(model_details)

In [ ]:
model_id

In [ ]:
x_train

In [ ]:
multiple_lin_reg.predict(x_train)